In [1]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c cafa-6-protein-function-prediction
!unzip /content/cafa-6-protein-function-prediction.zip
!rm /content/cafa-6-protein-function-prediction.zip

Mounted at /content/drive
  0% 0.00/91.2M [00:00<?, ?B/s]
100% 91.2M/91.2M [00:00<00:00, 1.10GB/s]
Archive:  /content/cafa-6-protein-function-prediction.zip
  inflating: IA.tsv                  
  inflating: Test/testsuperset-taxon-list.tsv  
  inflating: Test/testsuperset.fasta  
  inflating: Train/go-basic.obo      
  inflating: Train/train_sequences.fasta  
  inflating: Train/train_taxonomy.tsv  
  inflating: Train/train_terms.tsv   
  inflating: sample_submission.tsv   


In [2]:
%%capture
# !pip install autogluon.tabular[all]
!pip install obonet
!pip install pyvis
!pip install Bio

## Import Packages

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, GammaRegressor, Ridge
from sklearn.metrics import make_scorer, mean_squared_error, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# from keras_tuner import HyperModel
# from keras_tuner import RandomSearch
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import layers, models

from sklearn.feature_selection import f_classif
import obonet
from Bio import SeqIO
from transformers import AutoTokenizer, AutoModel

# from autogluon.tabular import TabularDataset, TabularPredictor

## Load Data

In [ ]:
train_terms = pd.read_csv("/content/Train/train_terms.tsv",sep="\t")
print(train_terms.shape)

(537027, 3)


In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "protein_embeddings.npy" #protein_ids.csv

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "seddiktrk/cafa6-protein-embeddings-esm2",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

In [ ]:
train_terms.head()

,EntryID,term,aspect
0,Q5W0B1,GO:0000785,C
1,Q5W0B1,GO:0004842,F
2,Q5W0B1,GO:0051865,P
3,Q5W0B1,GO:0006275,P
4,Q5W0B1,GO:0006513,P


### Make Protein Embeddings

In [4]:
import torch
from transformers import T5Tokenizer, T5EncoderModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# tokenizer = T5Tokenizer.from_pretrained(
#     "Rostlab/prot_t5_xl_half_uniref50-enc",
#     do_lower_case=False
# )

# model = T5EncoderModel.from_pretrained(
#     "Rostlab/prot_t5_xl_half_uniref50-enc"
# ).to(device)

tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D")
model = AutoModel.from_pretrained("facebook/esm2_t33_650M_UR50D").to(device)

if device.type == "cuda":
    model = model.half()

print("✅ ProtT5 loaded successfully!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ ProtT5 loaded successfully!


In [5]:
import re
def get_embeddings(seq):
    sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", seq)))]

    ids = tokenizer.batch_encode_plus(sequence_examples, add_special_tokens=True, padding="longest")

    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)

    # generate embeddings
    with torch.no_grad():
        embedding_repr = model(input_ids=input_ids,
                               attention_mask=attention_mask)

    # extract residue embeddings for the first ([0,:]) sequence in the batch and remove padded & special tokens ([0,:7])
    emb_0 = embedding_repr.last_hidden_state[0]
    emb_0_per_protein = emb_0.mean(dim=0)

    return emb_0_per_protein

In [6]:
fn = 'Train/train_sequences.fasta'
print("Sequence example:\n\n", next(iter(SeqIO.parse(fn, "fasta"))))
sequences = SeqIO.parse(fn, "fasta")
num_sequences = sum(1 for seq in sequences)
print()
print("Number of sequences in train:", num_sequences)

Sequence example:

 ID: sp|A0A0C5B5G6|MOTSC_HUMAN
Name: sp|A0A0C5B5G6|MOTSC_HUMAN
Description: sp|A0A0C5B5G6|MOTSC_HUMAN Mitochondrial-derived peptide MOTS-c OS=Homo sapiens OX=9606 GN=MT-RNR1 PE=1 SV=1
Number of features: 0
Seq('MRWQEMGYIFYPRKLR')

Number of sequences in train: 82404


In [7]:
import tqdm
fn = 'Train/train_sequences.fasta'

sequences = SeqIO.parse(fn, "fasta")

ids = []
embeds = np.zeros((num_sequences, 1024))
i = 0
for seq in tqdm.tqdm(sequences):
    ids.append(seq.id)
    embeds[i] = get_embeddings(str(seq.seq)).detach().cpu().numpy()
    i += 1




0it [00:01, ?it/s]


ValueError: could not broadcast input array from shape (1280,) into shape (1024,)

In [ ]:
np.save('/content/drive/MyDrive/Output/train_embeds.npy', embeds)
np.save('/content/drive/MyDrive/Output/train_ids.npy', np.array(ids))

In [ ]:
fn = 'Test/testsuperset.fasta'

sequences = SeqIO.parse(fn, "fasta")
num_sequences = sum(1 for seq in sequences)
print("Number of sequences in test:", num_sequences)
sequences = SeqIO.parse(fn, "fasta")


ids = []
embeds = np.zeros((num_sequences, 1024))
i = 0
for seq in tqdm.tqdm(sequences):
    ids.append(seq.id)
    embeds[i] = get_embeddings(str(seq.seq)).detach().cpu().numpy()
    i += 1




In [ ]:
np.save('/content/drive/MyDrive/Output/test_embeds.npy', embeds)
np.save('/content/drive/MyDrive/Output/test_ids.npy', np.array(ids))

#### Embed with Smaller Facebook Model

In [ ]:
# =========================================================
# ESM2-T33 embedding pipeline for FASTA sequences
# =========================================================

!pip install biopython transformers torch tqdm --quiet

import torch
import numpy as np
import re, gc
from tqdm import tqdm
from Bio import SeqIO
from transformers import AutoTokenizer, AutoModel

# -------------------------------
# 1. Device setup
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🧠 Using device:", device)

# -------------------------------
# 2. Load ESM2 model
# -------------------------------
model_name = "facebook/esm2_t33_650M_UR50D"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model = model.to(device)
model.eval()

if device.type == "cuda":
    model = model.half()  # saves memory
print("✅ ESM2 model loaded successfully!")

# -------------------------------
# 3. Embedding helper functions
# -------------------------------

MAX_LEN = 1024         # max sequence length to process at once
CHUNK_SIZE = 512       # chunk size for long sequences
OVERLAP = 64           # overlap to preserve context

def clean_sequence(seq: str) -> str:
    """Replace uncommon amino acids and remove invalid chars."""
    return re.sub(r"[^ACDEFGHIKLMNPQRSTVWYX]", "", re.sub(r"[UZOB]", "X", seq))

def embed_chunked(seq: str, chunk_size=CHUNK_SIZE, overlap=OVERLAP):
    """Handle long sequences by splitting into overlapping chunks."""
    seq = clean_sequence(seq)
    chunks = [
        seq[i:i+chunk_size]
        for i in range(0, len(seq), chunk_size - overlap)
    ]
    chunk_embs = []
    for chunk in chunks:
        emb = embed_single(chunk)
        chunk_embs.append(emb)
        del emb
        torch.cuda.empty_cache()
        gc.collect()
    return torch.stack(chunk_embs).mean(dim=0)

@torch.no_grad()
def embed_single(seq: str):
    """Compute mean protein embedding for a single sequence."""
    seq = clean_sequence(seq)
    seq = " ".join(list(seq))  # ESM expects space-separated residues
    tokens = tokenizer(
        seq,
        return_tensors="pt",
        add_special_tokens=True,
        padding=True,
        truncation=True,
        max_length=MAX_LEN
    )
    tokens = {k: v.to(device) for k, v in tokens.items()}
    # if device.type == "cuda":
    #     tokens = {k: v.half() for k, v in tokens.items()}

    out = model(**tokens)
    emb = out.last_hidden_state[0]
    emb_mean = emb.mean(dim=0)
    return emb_mean  # torch tensor (dim=1280)

def get_embedding(seq: str):
    """Choose chunked vs direct embedding."""
    seq = clean_sequence(seq)
    if len(seq) > MAX_LEN:
        return embed_chunked(seq)
    else:
        return embed_single(seq)

# -------------------------------
# 4. Process FASTA file
# -------------------------------
FASTA_PATH = "Train/train_sequences.fasta"

sequences = list(SeqIO.parse(FASTA_PATH, "fasta"))
num_sequences = len(sequences)
print(f"📄 Found {num_sequences} sequences.")

# Determine embedding dimension (ESM2-T33 has 1280)
EMB_DIM = 1280
embeds = np.zeros((num_sequences, EMB_DIM), dtype=np.float32)
ids = []

for i, record in enumerate(tqdm(sequences, desc="Embedding sequences")):
    seq_id = record.id
    seq_str = str(record.seq)
    emb = get_embedding(seq_str)
    embeds[i] = emb.detach().cpu().numpy()
    ids.append(seq_id)

    # free memory each iteration
    del emb
    torch.cuda.empty_cache()
    gc.collect()

print("✅ All embeddings computed!")

# -------------------------------
# 5. Save embeddings and IDs
# -------------------------------
np.save('/content/drive/MyDrive/Output/train_embeds.npy', embeds)
np.save('/content/drive/MyDrive/Output/train_ids.npy', np.array(ids))

print("💾 Saved embeddings to 'protein_embeddings.npy' and IDs to 'protein_ids.txt'")


🧠 Using device: cuda


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ ESM2 model loaded successfully!
📄 Found 82404 sequences.


Embedding sequences:  16%|█▌        | 13381/82404 [2:42:29<22:42:24,  1.18s/it]Exception ignored in: <function _xla_gc_callback at 0x7caa6b0445e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/lib/__init__.py", line 127, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
Embedding sequences:  16%|█▋        | 13396/82404 [2:42:41<13:16:19,  1.44it/s]Exception ignored in: <function _xla_gc_callback at 0x7caa6b0445e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/lib/__init__.py", line 127, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
Embedding sequences:  16%|█▋        | 13397/82404 [2:42:41<11:43:17,  1.64it/s]Exception ignored in: <function _xla_gc_callback at 0x7caa6b0445e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/lib/__init__.py", line 127, in _xla_gc_callback
    def _xla_gc_callbac

In [ ]:
from Bio import SeqIO
import numpy as np
import torch, gc
from tqdm import tqdm

# -------------------------------
# Load test FASTA
# -------------------------------
fn = "Test/testsuperset.fasta"

# Count sequences first
sequences = list(SeqIO.parse(fn, "fasta"))
num_sequences = len(sequences)
print("📄 Number of sequences in test:", num_sequences)

# -------------------------------
# Prepare containers
# -------------------------------
EMB_DIM = 1280   # ESM2-T33 embedding size
embeds = np.zeros((num_sequences, EMB_DIM), dtype=np.float32)
ids = []

# -------------------------------
# Generate embeddings
# -------------------------------
for i, seq in enumerate(tqdm(sequences, desc="Embedding test sequences")):
    seq_str = str(seq.seq)
    seq_id = seq.id
    emb = get_embedding(seq_str)   # uses same function from train pipeline
    embeds[i] = emb.detach().cpu().numpy()
    ids.append(seq_id)

    # clear memory
    del emb
    torch.cuda.empty_cache()
    gc.collect()

print("✅ Test embeddings computed!")

# -------------------------------
# Save results
# -------------------------------
np.save('/content/drive/MyDrive/Output/test_embeds.npy', embeds)
np.save('/content/drive/MyDrive/Output/test_ids.npy', np.array(ids))

print("💾 Saved test embeddings to 'test_embeddings.npy' and IDs to 'test_ids.txt'")


## EDA

## Save to Github

In [ ]:
!git config --global user.email "abishpius@gmail.com"
!git config --global user.name "Abish Pius"

In [ ]:
from google.colab import userdata
import requests

token = userdata.get('github')
repo_name = "Cafa-6-protein-function"
username = "abishpius"
message = "Kaggle Competition Cafa 6 Protein Function"
notebook_name = "Cafa_6_Protein_Function.ipynb"

headers = {
    "Authorization": f"token {token}",
    "Accept": "application/vnd.github+json"
}

# === CHECK IF REPO EXISTS ===
check_url = f"https://api.github.com/repos/{username}/{repo_name}"
check_resp = requests.get(check_url, headers=headers)

if check_resp.status_code == 200:
    print(f"❌ Repo '{repo_name}' already exists at: https://github.com/{username}/{repo_name}")
else:
    # === CREATE REPO ===
    create_url = "https://api.github.com/user/repos"
    payload = {
        "name": repo_name,
        "auto_init": True,
        "private": False  # change to True if you want a private repo
    }

    create_resp = requests.post(create_url, headers=headers, json=payload)

    if create_resp.status_code == 201:
        print(f"✅ Repo '{repo_name}' created successfully.")

        # === UPDATE README.md ===
        import base64

        update_url = f"https://api.github.com/repos/{username}/{repo_name}/contents/README.md"
        readme_text = ""
        encoded = base64.b64encode(readme_text.encode()).decode()
        update_payload = {
            "message": message,
            "content": encoded,
            "sha": requests.get(update_url, headers=headers).json()["sha"]
        }

        update_resp = requests.put(update_url, headers=headers, json=update_payload)

        if update_resp.status_code == 200 or update_resp.status_code == 201:
            print("✅ README.md updated with custom text.")
        else:
            print("⚠️ Failed to update README.md:", update_resp.json())

    else:
        print("❌ Repo creation failed:", create_resp.json())

✅ Repo 'Cafa-6-protein-function' created successfully.
✅ README.md updated with custom text.


In [ ]:
!git clone https://{token}@github.com/{username}/{repo_name}.git

# === MOVE NOTEBOOK ===
import shutil
shutil.copy(f"/content/drive/MyDrive/Colab Notebooks/{notebook_name}", f"/content/{repo_name}/{notebook_name}")

# === COMMIT TO REPO ===
%cd {repo_name}
!git add {notebook_name}
!git commit -m "T5 embedding generation attempt"
!git push

Cloning into 'Cafa-6-protein-function'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), done.
/content/Cafa-6-protein-function
[main de0af6c] T5 embedding generation attempt
 1 file changed, 1 insertion(+)
 create mode 100644 Cafa_6_Protein_Function.ipynb
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 13.93 KiB | 4.64 MiB/s, done.
Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/abishpius/Cafa-6-protein-function.git
   a2555ea..de0af6c  main -> main
